In [4]:
import os
import cv2
import json
import numpy as np
import subprocess

# Path utama untuk OpenPose dan direktori gambar
openpose_path = os.path.abspath("openpose")  # Path ke OpenPose
image_dir = os.path.abspath("../images")  # Folder gambar input
json_output_dir = os.path.abspath("../output_json")  # Folder JSON hasil OpenPose
cropped_dir = os.path.abspath("../cropped_images")  # Folder hasil crop

# Buat folder jika belum ada
os.makedirs(image_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(cropped_dir, exist_ok=True)

# Ambil file gambar dari folder `images/`
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

if not image_files:
    print("⚠️ Tidak ada gambar yang valid di folder `images/`. Silakan unggah file JPG atau PNG!")
    exit()

# Loop untuk memproses semua gambar dalam folder `images/`
for image_file in image_files:
    input_image_path = os.path.join(image_dir, image_file)
    file_name, file_ext = os.path.splitext(image_file)

    output_json_path = os.path.join(json_output_dir, file_name + '_keypoints.json')

    # Jalankan OpenPose dengan `cwd=openpose_path` (tanpa menyimpan gambar hasil OpenPose)
    cmd = [
        "bin\\OpenPoseDemo.exe",
        "--image_dir", image_dir,
        "--write_json", json_output_dir,
        "--display", "0",
        "--render_pose", "0",
        "--model_pose", "BODY_25",
        "--number_people_max", "1"
    ]

    print(f"🚀 Memproses gambar {image_file} dengan OpenPose...")

    result = subprocess.run(cmd, shell=True, capture_output=True, text=True, cwd=openpose_path)

    if result.returncode != 0:
        print(f"❌ Gagal memproses gambar {image_file} dengan OpenPose!")
        continue

    # Baca hasil JSON dari OpenPose
    if not os.path.exists(output_json_path):
        print(f"⚠️ JSON hasil deteksi tidak ditemukan untuk {image_file}!")
        continue

    with open(output_json_path, 'r') as f:
        pose_data = json.load(f)
        if not pose_data['people']:
            print(f"⚠️ Tidak ada orang terdeteksi pada {image_file}.")
            continue
        keypoints = pose_data['people'][0]['pose_keypoints_2d']

    # Ambil koordinat x, y untuk semua keypoints
    def get_coords(index):
        x = keypoints[index * 3]
        y = keypoints[index * 3 + 1]
        confidence = keypoints[index * 3 + 2]
        return (int(x), int(y)) if confidence > 0 else None

    body_points = [get_coords(i) for i in range(25)]
    valid_points = [p for p in body_points if p is not None]

    if not valid_points:
        print(f"⚠️ Tidak ada titik tubuh yang valid pada {image_file}.")
        continue

    # Cari bounding box (xmin, ymin, xmax, ymax)
    x_coords, y_coords = zip(*valid_points)
    xmin, xmax = min(x_coords), max(x_coords)
    ymin, ymax = min(y_coords), max(y_coords)

    # Tambahkan margin ekstra (10% dari bounding box)
    margin_x = int((xmax - xmin) * 0.1)
    margin_y = int((ymax - ymin) * 0.1)

    xmin, ymin = max(0, xmin - margin_x), max(0, ymin - margin_y)
    xmax, ymax = min(xmax + margin_x, 9999), min(ymax + margin_y, 9999)  # 9999 untuk menjaga batas atas

    # Tentukan ukuran bounding box agar tetap 1:1
    box_size = max(xmax - xmin, ymax - ymin)

    # Hitung posisi baru agar tetap dalam batas gambar
    x_center, y_center = (xmin + xmax) // 2, (ymin + ymax) // 2
    xmin, ymin = x_center - box_size // 2, y_center - box_size // 2
    xmax, ymax = x_center + box_size // 2, y_center + box_size // 2

    # Pastikan bounding box tetap dalam ukuran gambar
    img = cv2.imread(input_image_path)
    h, w, _ = img.shape

    # Hitung padding yang diperlukan
    pad_top, pad_bottom, pad_left, pad_right = 0, 0, 0, 0

    if xmin < 0:
        pad_left = abs(xmin)
        xmin = 0
    if xmax > w:
        pad_right = xmax - w
        xmax = w
    if ymin < 0:
        pad_top = abs(ymin)
        ymin = 0
    if ymax > h:
        pad_bottom = ymax - h
        ymax = h

    # Crop gambar sesuai bounding box yang diperbaiki
    cropped_img = img[ymin:ymax, xmin:xmax]

    # Tambahkan padding abu-abu agar tetap 1:1 dan objek di tengah
    cropped_img = cv2.copyMakeBorder(
        cropped_img, pad_top, pad_bottom, pad_left, pad_right,
        cv2.BORDER_CONSTANT, value=(128, 128, 128)  # Warna abu-abu
    )

    # Resize gambar ke ukuran standar 256x256
    final_size = (256, 256)
    resized_img = cv2.resize(cropped_img, final_size)

    # Simpan hasil crop
    cropped_image_path = os.path.join(cropped_dir, f"{file_name}_cropped{file_ext}")
    cv2.imwrite(cropped_image_path, resized_img)

    print(f"✅ Gambar hasil crop disimpan: {cropped_image_path}")

print("🎉 Semua gambar berhasil diproses dan disimpan di folder `cropped_images/`.")


🚀 Memproses gambar flipped_duduk2.jpg dengan OpenPose...
✅ Gambar hasil crop disimpan: F:\cropped_images\flipped_duduk2_cropped.jpg
🎉 Semua gambar berhasil diproses dan disimpan di folder `cropped_images/`.
